In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/gdsc-hics-machinelearning-hackathon/train.csv', index_col='p_id')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### handle missing values

In [ ]:
df['glucose_concentration'] = df['glucose_concentration'].fillna(df['glucose_concentration'].mean())

In [ ]:
df.info()

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
df[['serum_insulin']] = imputer.fit_transform(df[['serum_insulin']])

In [ ]:
type(df)

In [ ]:
df.head()

### feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.loc[:, df.columns != 'diabetes'].values)

cols = df.columns[:-1]
df[cols] = scaled_features
# scaled_features_df = pd.DataFrame(scaled_features, index=df.index, columns=cols)


# df_final = pd.concat([scaled_features_df, df['diabetes']])

In [ ]:
df.head()

### visualizing data

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(df, alpha=0.2, figsize=(12, 12), diagonal="kde");

In [ ]:
# sns.heatmap(data = df, annot=True)

## building the model

### cross validation

In [ ]:
df.head()

In [ ]:
X = df.drop('diabetes', axis='columns').values
y = df['diabetes'].values

### splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

svc = SVC()
lr = LogisticRegression()
rfc = RandomForestClassifier()


# from sklearn.pipeline import Pipeline

# steps = [('imputation', Imputer(missing_values='?', strategy='most_frequent')),
#         ('scaler', StandardScaler()),
#         ('svc', svc)]

# pipeline = Pipeline(steps)

# from sklearn.model_selection import GridSearchCV

# parameters_svc = {'gamma': np.linspace(0.01, 1.5, 15)}
# parameters_lr = {}
# parameters_rfc = {}

# svc_cv = GridSearchCV(pipeline, parameters)

In [ ]:
from sklearn.ensemble import VotingClassifier

classifiers = [('Support Vector Classifier', svc), ('Logistic Regression', lr), ('Random Forest Classifier', rfc)]
vc = VotingClassifier(estimators=classifiers)

# vc.fit(X_train, y_train)
# y_pred = vc.predict(X_test)

# vc.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

print(cross_val_score(vc, X_train, y_train))
vc.fit(X_train, y_train)
y_pred = vc.predict(X_test)

print('final score is {:.3f}'.format(vc.score(X_test, y_test)))

## evaluating the model

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

### previewing test set

In [ ]:
df_test = pd.read_csv('../input/gdsc-hics-machinelearning-hackathon/test.csv')
df_test.head()

In [ ]:

# from sklearn.pipeline import Pipelinedf_test.shape

#### preparing for submission

In [ ]:
from sklearn.pipeline import Pipeline

# steps = [('imputation', SimpleImputer(missing_values=np.nan, strategy='mean')),
#         ('scaler', StandardScaler()),
#         ('vc', vc)]

# pipeline = Pipeline(steps)

# pipeline.fit_transform(df_test)
features = df_test.drop("p_id", axis=1)
scaled_features = scaler.fit_transform(features.values)
features['Expected'] = vc.predict(scaled_features)


In [ ]:
features

In [ ]:
df_submit = pd.DataFrame(data={'p_id': df_test['p_id'], 'diabetes': features['Expected']})

In [ ]:
df_submit.head()

In [ ]:
df_submit.to_csv('submission.csv', index=False)

In [ ]:
df_sub = pd.read_csv('./submission.csv')
df_sub.head()